In [1]:
# !pip install -U langchain
# !pip install -U langchain-openai
# !pip install -U langchain_core

- https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/

In [19]:
from dotenv import load_dotenv
import os
from IPython.display import Image
assert load_dotenv()

In [20]:
os.environ['LANGCHAIN_PROJECT'] = '0929'

In [7]:
# load llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# tools
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)
    
print(get_word_length.invoke("abc"))

tools = [get_word_length]
llm_with_tools = llm.bind_tools(tools)

3


### prompt template

In [13]:
from rich.pretty import pprint

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

pprint(prompt)

ChatPromptTemplate(
│   input_variables=['agent_scratchpad', 'input'],
│   input_types={
│   │   'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.FunctionMessageChunk, Tag(tag='FunctionMessageChunk')], typing.Annotated[langchain_core.messages.tool.ToolMessageChunk, Tag(tag='ToolMessageChunk')]], FieldInfo(annotation=NoneType, required=True, discriminator=Discriminator(discriminator=<function _get_type at 0x7e42c61ae700>, custom_error_type=None, custom_error_message=None, custom_error_context=None))]]
│   },
│   partial_variables={},
│   messages=[
│   │   SystemMessagePromptTemplate(
│   │   │   prompt=PromptTemplate(
│   │   │   │   input_variables=[],
│   │   │   │   input_types={},
│   │   │   │   partial_variables={},
│   │   │   │   template="You are very powerful assistant, but don't know current events"
│   │   │   ),
│   │   │   additional_kwargs={}
│   │   ),
│   │   HumanMessagePromptTemplate(
│   │   │   prompt=PromptTemplate(
│   │   │   │   input_variables=['input'],
│   │   │   │   input_types={},
│   │   │   │   partial_variables={},
│   │   │   │   template='{input}'
│   │   │   ),
│   │   │   additional_kwargs={}
│   │   ),
│   │   MessagesPlaceholder(variable_name='agent_scratchpad')
│   ]
)

### agent & agent executor

In [15]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [18]:
Image(url='https://miro.medium.com/v2/resize:fit:4800/format:webp/1*CD2Svi5BUZaG-d-KWeU3ug.png', width=500)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

- `intermediate steps`: Denotes preceding agent actions and their corresponding outputs, organized as a List of Tuples `[AgentAction, Any].`
- The `AgentExecutor` is tasked with utilizing the Agent **until the final output is attained**. Consequently, it employs the Agent to obtain the next action, executes the returned action **iteratively**, and continues this process until a conclusive answer is generated for the given input.
- messages list
    - System: You are very powerful assistant, but don't know current events
    - Human: How many letters in the word eudca
    - AI:
    ```
    [
      {
        "name": "get_word_length",
        "args": {
          "word": "eudca"
        },
        "id": "call_7iFwbEl7nHS3K3scMOF6ubH5",
        "type": "tool_call"
      }
    ]
    ```
    - Tool:
    ```
    5
    ```

In [22]:
agent_executor.invoke({"input": "How many letters in the word eudca"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


{'input': 'How many letters in the word eudca',
 'output': 'The word "eudca" has 5 letters.'}

### with `chat_history`

In [24]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name='chat_history'),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [25]:
pprint(prompt)

ChatPromptTemplate(
│   input_variables=['agent_scratchpad', 'chat_history', 'input'],
│   input_types={
│   │   'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.FunctionMessageChunk, Tag(tag='FunctionMessageChunk')], typing.Annotated[langchain_core.messages.tool.ToolMessageChunk, Tag(tag='ToolMessageChunk')]], FieldInfo(annotation=NoneType, required=True, discriminator=Discriminator(discriminator=<function _get_type at 0x7e42c61ae700>, custom_error_type=None, custom_error_message=None, custom_error_context=None))]],
│   │   'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.FunctionMessageChunk, Tag(tag='FunctionMessageChunk')], typing.Annotated[langchain_core.messages.tool.ToolMessageChunk, Tag(tag='ToolMessageChunk')]], FieldInfo(annotation=NoneType, required=True, discriminator=Discriminator(discriminator=<function _get_type at 0x7e42c61ae700>, custom_error_type=None, custom_error_message=None, custom_error_context=None))]]
│   },
│   partial_variables={},
│   messages=[
│   │   SystemMessagePromptTemplate(
│   │   │   prompt=PromptTemplate(
│   │   │   │   input_variables=[],
│   │   │   │   input_types={},
│   │   │   │   partial_variables={},
│   │   │   │   template='You are very powerful assistant, but bad at calculating lengths of words.'
│   │   │   ),
│   │   │   additional_kwargs={}
│   │   ),
│   │   MessagesPlaceholder(variable_name='chat_history'),
│   │   HumanMessagePromptTemplate(
│   │   │   prompt=PromptTemplate(
│   │   │   │   input_variables=['input'],
│   │   │   │   input_types={},
│   │   │   │   partial_variables={},
│   │   │   │   template='{input}'
│   │   │   ),
│   │   │   additional_kwargs={}
│   │   ),
│   │   MessagesPlaceholder(variable_name='agent_scratchpad')
│   ]
)

In [26]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [27]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
input1 = "how many letters in the word ddadgawedaewu?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'ddadgawedaewu'}`


13The word "ddadgawedaewu" has 13 letters.

> Finished chain.


In [30]:
result

{'input': 'how many letters in the word ddadgawedaewu?',
 'chat_history': [HumanMessage(content='how many letters in the word ddadgawedaewu?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The word "ddadgawedaewu" has 13 letters.', additional_kwargs={}, response_metadata={})],
 'output': 'The word "ddadgawedaewu" has 13 letters.'}

In [29]:
chat_history

[HumanMessage(content='how many letters in the word ddadgawedaewu?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The word "ddadgawedaewu" has 13 letters.', additional_kwargs={}, response_metadata={})]

In [31]:
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...
No, "ddadgawedaewu" is not a real word. It appears to be a random combination of letters.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word ddadgawedaewu?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The word "ddadgawedaewu" has 13 letters.', additional_kwargs={}, response_metadata={})],
 'output': 'No, "ddadgawedaewu" is not a real word. It appears to be a random combination of letters.'}